In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import random
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')


# Step 1: Create Dataset
def create_body_type_dataset_with_index(num_samples=2000):
    heights = []
    weights = []
    ages = []
    body_type_indices = []

    for _ in range(num_samples):
        age = random.randint(5, 80)

        if age < 18:
            height = random.randint(100, 180)
            weight = random.randint(20, 80)
        else:
            height = random.randint(120, 210)
            weight = random.randint(40, 200)

        bmi = weight / (height / 100) ** 2

        if age > 60:
            bmi += 1
        elif age < 18:
            bmi -= 1

        if bmi < 16:
            body_type_index = 0
        elif 16 <= bmi < 18.5:
            body_type_index = 1
        elif 18.5 <= bmi < 25:
            body_type_index = 2
        elif 25 <= bmi < 30:
            body_type_index = 3
        elif 30 <= bmi < 35:
            body_type_index = 4
        else:
            body_type_index = 5

        heights.append(height)
        weights.append(weight)
        ages.append(age)
        body_type_indices.append(body_type_index)

    data = {
        'Height': heights,
        'Weight': weights,
        'Age': ages,
        'BodyTypeIndex': body_type_indices
    }

    return pd.DataFrame(data)


# Step 2: Train and Save Model
def train_and_save_model():
    df = create_body_type_dataset_with_index(2000)
    X = df[['Height', 'Weight', 'Age']]
    y = df['BodyTypeIndex']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Pipeline with StandardScaler and SVC with probability enabled
    model = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1.0, gamma='scale', probability=True))
    model.fit(X_train, y_train)

    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred)
    val_accuracy = accuracy_score(y_test, y_test_pred)

    # Validation log loss (probability required)
    from sklearn.metrics import log_loss
    y_test_proba = model.predict_proba(X_test)
    val_loss = log_loss(y_test, y_test_proba)

    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation Log Loss: {val_loss:.4f}\n")

    # Classification Report
    print("Classification Report:\n", classification_report(y_test, y_test_pred))

    # Save model using pickle
    with open('svm_body_type_model.pkl', 'wb') as file:
        pickle.dump(model, file)

    print("✅ Model saved as 'svm_body_type_model.pkl'")


# Step 3: Load and Use Model
def predict_body_type():
    try:
        with open('svm_body_type_model.pkl', 'rb') as file:
            model = pickle.load(file)

        height = float(input("Enter your height in centimeters: "))
        weight = float(input("Enter your weight in kilograms: "))
        age = int(input("Enter your age: "))

        input_data = [[height, weight, age]]
        prediction = model.predict(input_data)[0]

        index_to_bodytype = {
            0: "Extremely Weak", 1: "Weak", 2: "Normal",
            3: "Overweight", 4: "Obesity", 5: "Extreme Obesity"
        }

        print(f"Your BodyTypeIndex is: {prediction}")
        print(f"Your Body Type is: {index_to_bodytype[prediction]}")

    except FileNotFoundError:
        print("Model file not found. Please train the model first by running train_and_save_model().")
    except ValueError:
        print("Invalid input. Please enter numeric values for height, weight, and age.")


# Main Execution
if __name__ == "__main__":
    # Run once to train and save the model
    train_and_save_model()

    # Run user input for prediction
    predict_body_type()


Training Accuracy: 0.9169
Validation Accuracy: 0.9125
Validation Log Loss: 0.2438

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94        31
           1       0.60      0.23      0.33        13
           2       0.74      0.96      0.84        48
           3       0.85      0.87      0.86        46
           4       1.00      0.52      0.68        31
           5       0.97      1.00      0.98       231

    accuracy                           0.91       400
   macro avg       0.85      0.75      0.77       400
weighted avg       0.91      0.91      0.90       400

✅ Model saved as 'svm_body_type_model.pkl'
Enter your height in centimeters: 170
Enter your weight in kilograms: 70
Enter your age: 23
Your BodyTypeIndex is: 2
Your Body Type is: Normal


In [ ]:
from google.colab import files
files.download('svm_body_type_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>